# Stochastic Volatility

[Stochastic volatility](https://en.wikipedia.org/wiki/Stochastic_volatility) is the measure of variance of a stochastic process, often used in finance to predict the volatility of stocks and other securities.  Quantifying volatility allows for more accurate predictions and forecasts.  In this tutorial, we'll demonstrate how to build a simple multivariate stochastic volatility following the example in Yu & Meyer \[1\].

Our model is a constant correlation volatility model, which means that there is a fixed underlying volatility we are trying to measure.  The dynamic process is described as follows:

$$ y = \mathrm{diag}(\mathrm{exp}(h_t/2)) \epsilon_t$$
$$ h_{t+1} = \mu + \Phi(h_t - \mu) + \eta_t$$

where $\Phi$ is some transition matrix,

$$ \eta_t \sim N(0, \Sigma_\eta)$$
$$ \epsilon_t \sim N(0, \Sigma_\epsilon)$$
$$ \Sigma_\eta = \begin{bmatrix} 
1 & \rho \\
\rho & 1 
\end{bmatrix}$$
$$ \Sigma_\epsilon = \begin{bmatrix} 
\sigma_1 & 0 \\
0 & \sigma_2
\end{bmatrix}$$
are sampled i.i.d.  If we factor out the $\mathrm{exp}(h_t/2)$ and take the logarithm of both sides, we can write the expression in terms of additive noise:

$$ \log y_t = \frac{h_t}{2} + \log\epsilon_t$$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.distributions import constraints

import pyro
import pyro.distributions as dist
from pyro.infer.autoguide import AutoDelta, AutoMultivariateNormal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

pyro.set_rng_seed(1)
assert pyro.__version__.startswith('0.4.1')

We'll generate some synthetic data using the model described above for 100 timesteps.

In [55]:
def sequential_model(num_samples=10, timesteps=500, state_dim=2, init_dist=None):
    """
    Generate data of shape: (samples, timesteps, state_dim)
    where the generative model is defined by:
        y = exp(h/2) * eps
        h_{t+1} = mu + Phi (h_t - mu) + eta_t
    where eps and eta are sampled iid from a MVN distribution
    """
    ys = []
    mu_trans = torch.zeros(state_dim)
    L_trans = pyro.param('L_eta', 0.2 * torch.eye(state_dim, state_dim))
    mu_obs = pyro.param('mu_gamma', torch.zeros(state_dim))
    L_obs = pyro.param('L_gamma', 0.2 * torch.eye(state_dim, state_dim))
    transition = pyro.param('phi', 0.2 * torch.randn(state_dim, state_dim))
    obs = torch.eye(state_dim)
    if init_dist is None:
        z = torch.zeros(num_samples, state_dim)
    else:
        z = pyro.sample('z_0', init_dist)  
#     with pyro.plate('samples', num_samples):
    trans_dist = dist.MultivariateNormal(mu_trans, scale_tril=L_trans).expand((num_samples,))
    obs_dist = dist.MultivariateNormal(mu_obs, scale_tril=L_obs).expand((num_samples,))
    transition = transition.expand(num_samples, -1, -1)

    for i in range(timesteps):
        trans_noise = pyro.sample('trans_noise', trans_dist)
        z = z.unsqueeze(1).bmm(transition).squeeze(1) + trans_noise
        # add observation noise
        obs_noise = pyro.sample('obs_noise', obs_dist)
        y = z @ obs + obs_noise
        ys.append(y)
    data = torch.stack(ys, 1)
    assert data.shape == (num_samples, timesteps, state_dim)
    return data

In [56]:
with torch.no_grad():
    data = sequential_model()
print(data.shape)

torch.Size([10, 500, 2])


To perform inference that parallelizes across time series, we can use the `GaussianHMM` distribution.

In [57]:
def hmm_model(data):
    # TODO put priors over the params here
    state_dim = data.shape[-1]
    with pyro.plate(len(data)):
        mu = pyro.param('mu', torch.zeros(state_dim))
        L = pyro.param('L', 0.1 * torch.eye(state_dim), constraint=constraints.lower_cholesky)
        init_dist = dist.MultivariateNormal(mu, scale_tril=L)

        L_eta = pyro.param('L_eta', 0.4 * torch.eye(state_dim), constraint=constraints.lower_cholesky)
        mu_eta = torch.zeros(state_dim)
        trans_matrix = pyro.param('phi', 0.5 * torch.eye(state_dim))
        trans_dist = dist.MultivariateNormal(mu_eta, scale_tril=L_eta)

        mu_gamma = pyro.param('mu_gamma', torch.zeros(state_dim))
        L_gamma = pyro.param('L_gamma', 0.5 * torch.eye(state_dim), constraint=constraints.lower_cholesky)
        obs_matrix = torch.eye(state_dim, state_dim)
        # latent state is h_t - mu
        obs_dist = dist.MultivariateNormal(-mu_gamma, scale_tril=L_gamma)

        hmm_dist = dist.GaussianHMM(init_dist, trans_matrix, trans_dist, obs_matrix, obs_dist)
        pyro.sample('obs', hmm_dist, obs=data)

In [61]:
pyro.clear_param_store()
guide = AutoMultivariateNormal(hmm_model)
svi = SVI(hmm_model, guide, Adam({'lr': 0.01}), Trace_ELBO())
for i in range(500):
    loss = svi.step(data)
    if i % 10 == 0:
        logging.info('epoch {}: {: 4f}'.format(i, loss))
for k, v in pyro.get_param_store().items():
    print(k, v.detach().cpu().numpy())

RuntimeError: AutoMultivariateNormal found no latent variables; Use an empty guide instead
Trace Shapes:
 Param Sites:
Sample Sites:

Now let's compare the posterior computed by the Diagonal Normal guide vs the Multivariate Normal guide.  Note that the multivariate distribution is more dispresed than the Diagonal Normal.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
fig.suptitle("Cross-sections of the Posterior Distribution", fontsize=16)
sns.kdeplot(svi_samples["bA"], svi_samples["bR"], ax=axs[0], label="HMC")
sns.kdeplot(svi_mvn_samples["bA"], svi_mvn_samples["bR"], ax=axs[0], shade=True, label="SVI (Multivariate Normal)")
axs[0].set(xlabel="bA", ylabel="bR", xlim=(-2.5, -1.2), ylim=(-0.5, 0.1))
sns.kdeplot(svi_samples["bR"], svi_samples["bAR"], ax=axs[1], label="SVI (Diagonal Normal)")
sns.kdeplot(svi_mvn_samples["bR"], svi_mvn_samples["bAR"], ax=axs[1], shade=True, label="SVI (Multivariate Normal)")
axs[1].set(xlabel="bR", ylabel="bAR", xlim=(-0.45, 0.05), ylim=(-0.15, 0.8))
handles, labels = axs[1].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right');

## References
[1] Yu and Meyer. [Multivariate Stochastic Volatility Models](https://pdfs.semanticscholar.org/fccc/6f4ee933d4330eabf377c08f8b2650e1f244.pdf). 2006.